In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine,text

In [2]:
order_df=pd.read_csv('data/orders.csv')

In [3]:
order_df.drop('eval_set',inplace=True,axis=1)

In [4]:
order_df.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0


In [5]:
aisle_df=pd.read_csv('data/aisles.csv')

In [6]:
aisle_df.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [7]:
department_df=pd.read_csv('data/departments.csv')

In [8]:
department_df.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [9]:
order_products_df=pd.read_csv('data/order_products.csv')

In [10]:
order_products_df.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [11]:
product_df=pd.read_csv('data/products.csv')

In [12]:
product_df.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [ ]:
# Connection
try:
    conn=psycopg2.connect(dbname='ecomerce_analysis',user='postgres',password='',port='5432')
except:
    print('Connection was unsuccessful')

In [18]:
curr=conn.cursor()

In [ ]:
engine= create_engine('postgresql+psycopg2://postgres:@localhost/ecomerce_analysis')

In [20]:
curr.execute("""
create table aisles(
    aisle_id int primary key,
    aisle varchar(255) 
)
""")

In [21]:
curr.execute("""
create table departments(
    department_id int primary key,
    department varchar(255) 
)
""")

In [22]:
curr.execute("""
create table products(
    product_id int primary key,
    product_name varchar(255),
    aisle_id int,
    department_id int,
    foreign key (aisle_id) references aisles(aisle_id),
    foreign key (department_id) references departments(department_id)
)
""")

In [23]:
curr.execute("""
create table orders(
    order_id int primary key,
    user_id int,
    order_number int,
    order_dow int,
    order_hour_of_day int,
    days_since_prior_order int
)
""")

In [24]:
curr.execute("""
create table order_products(
    order_id int,
    product_id int,
    add_to_cart_order int,
    reordered int,
    primary key (order_id,product_id),
    foreign key (order_id) references orders(order_id),
    foreign key (product_id) references products(product_id)
)
""")

In [25]:
conn.commit()

In [26]:
aisle_df.to_sql('aisles',con=engine,if_exists='append',index=False)

134

In [27]:
department_df.to_sql('departments',con=engine,if_exists='append',index=False)

21

In [28]:
product_df.to_sql('products',con=engine,if_exists='append',index=False)

688

In [30]:
with engine.begin() as conn:
    conn.execute(text("DELETE FROM orders"))
    order_df.to_sql('orders', con=conn, if_exists='append', index=False)

In [29]:
order_products_df.to_sql('order_products',con=engine,if_exists='append',index=False)

28